<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-2">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-3">Main</a></span></li><li><span><a href="#Add-tumour-subtypes" data-toc-modified-id="Add-tumour-subtypes-4">Add tumour subtypes</a></span><ul class="toc-item"><li><span><a href="#ATRT-and-MB" data-toc-modified-id="ATRT-and-MB-4.1">ATRT and MB</a></span></li><li><span><a href="#GBM-subgroups" data-toc-modified-id="GBM-subgroups-4.2">GBM subgroups</a></span><ul class="toc-item"><li><span><a href="#CBTN" data-toc-modified-id="CBTN-4.2.1">CBTN</a></span></li><li><span><a href="#ICGC" data-toc-modified-id="ICGC-4.2.2">ICGC</a></span></li></ul></li><li><span><a href="#EPN-subgroups" data-toc-modified-id="EPN-subgroups-4.3">EPN subgroups</a></span></li></ul></li><li><span><a href="#NBL-subgroups" data-toc-modified-id="NBL-subgroups-5">NBL subgroups</a></span></li><li><span><a href="#LGG-subgroups" data-toc-modified-id="LGG-subgroups-6">LGG subgroups</a></span></li></ul></div>

# Dependencies

In [1]:
library(maftools)

# Paths

In [2]:
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Manifests/"
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Plots/"

# Main

In [3]:
load(paste0(datapath, "ESTIMATE/estimate_manifest_primary_clean_final.RData"))

In [4]:
table(estimate_manifest_primary_clean$group)


  CBTN   ICGC TARGET   TCGA 
   581    196    149   2452 

In [5]:
metadata <- estimate_manifest_primary_clean

# Add tumour subtypes

In [6]:
metadata$tumour_subtype <- NA

## ATRT and MB 

from ICGC or gene expression clustering

In [7]:
metadata$tumour_subtype[ metadata$cohort == "MB"] <- metadata$disease_type[ metadata$cohort == "MB"]

In [8]:
table(metadata$tumour_subtype)


MB_GR34  MB_SHH  MB_WNT 
    180      17      11 

In [9]:
# some cleanup
metadata$tumour_subtype[metadata$tumour_subtype == "MB_Group3"] <- "MB_GR34"
metadata$tumour_subtype[metadata$tumour_subtype == "MB_Group4"] <- "MB_GR34"

In [10]:
table(metadata$tumour_subtype)


MB_GR34  MB_SHH  MB_WNT 
    180      17      11 

In [11]:
metadata$tumour_subtype[ metadata$cohort == "ATRT"] <- metadata$disease_type[ metadata$cohort == "ATRT"]

## GBM subgroups

### CBTN

The following is to add variantfilenames to oncokb annotated combined_maf using CBTTC_summary_mutations_MuTect2.txt (generated on cavatica)

In [12]:
CBTN_mafsummary <- read.table(file = paste0(datapath,"snv/CBTTC_mafsummary_overlapped_3VAF.txt"),
                              sep = "\t", check.names = F, header = TRUE,  stringsAsFactors = FALSE)

In [13]:
head(CBTN_mafsummary)

,Tumor_Sample_Barcode,Frame_Shift_Del,Frame_Shift_Ins,In_Frame_Del,In_Frame_Ins,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,total,sample_id
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
1,BS_8AY2GM4G,21,21,0,1,9555,572,5,10175,7316-2085
2,BS_20TBZG09,17,31,0,1,9213,534,5,9801,7316-515
3,BS_02YBZSBY,9,165,0,0,8170,470,1,8815,7316-2189
4,BS_D0T6V861,9,154,0,0,8018,457,1,8639,7316-2189
5,BS_VW4XN9Y7,3,111,0,1,5563,266,7,5951,7316-2640
6,BS_F0GNWEJJ,42,14,9,0,717,25,0,807,7316-3311


In [14]:
dim(CBTN_mafsummary)

[1] 796  10

In [15]:
mymaf_CBTN <- read.maf(maf =  paste0(datapath,"snv/CBTTC_maf_overlapped_3VAF.maf"),
                       useAll = TRUE, verbose = FALSE,
                       vc_nonSyn = c("Frame_Shift_Del", "Frame_Shift_Ins", "Nonsense_Mutation","Nonstop_Mutation", 
                                     "In_Frame_Del","In_Frame_Ins", "Missense_Mutation"))

In [16]:
mymaf_CBTN@summary

ID,summary,Mean,Median
<chr>,<chr>,<dbl>,<dbl>
NCBI_Build,GRCh38,NA,NA
Center,Mutect2,NA,NA
Samples,842,NA,NA
nGenes,13511,NA,NA
Frame_Shift_Del,845,1.004,0
Frame_Shift_Ins,769,0.913,0
In_Frame_Del,160,0.190,0
In_Frame_Ins,30,0.036,0
Missense_Mutation,50771,60.298,4


In [17]:
CBTN_data_maf <- mymaf_CBTN@data

In [18]:
CBTN_data_maf_sampleid <- merge(CBTN_mafsummary[,c("sample_id", "Tumor_Sample_Barcode")],
                                CBTN_data_maf,by = "Tumor_Sample_Barcode")

In [19]:
length(unique(CBTN_data_maf_sampleid$Tumor_Sample_Barcode))

[1] 796

Subset  snv dfs to include only samples in estimate df

In [20]:
CBTN_data_maf_primary <- CBTN_data_maf_sampleid[CBTN_data_maf_sampleid$sample_id %in% metadata$sample_id,]

In [21]:
length(unique(CBTN_data_maf_primary$Tumor_Sample_Barcode))

[1] 470

In [22]:
head(CBTN_data_maf_primary)

,Tumor_Sample_Barcode,sample_id,chrpos,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,⋯,LEVEL_1,LEVEL_2A,LEVEL_2B,LEVEL_3A,LEVEL_3B,LEVEL_4,LEVEL_R1,Highest_level,citations,t_VAF
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<dbl>
1,BS_007JTNB8,7316-2558,chr19:10553056,ATG4D,84971,Mutect2,GRCh38,chr19,10553056,10553056,⋯,NA,NA,,NA,,,NA,,,0.3736264
55,BS_01DQH017,7316-3299,chr1:36469717,CSF3R,1441,Mutect2,GRCh38,chr1,36469717,36469717,⋯,NA,NA,,NA,,,NA,,,0.2428571
56,BS_01DQH017,7316-3299,chr19:57444698,ZNF749,388567,Mutect2,GRCh38,chr19,57444698,57444698,⋯,NA,NA,,NA,,,NA,,,0.1355932
57,BS_01DQH017,7316-3299,chr9:136723199,FAM69B,138311,Mutect2,GRCh38,chr9,136723199,136723199,⋯,NA,NA,,NA,,,NA,,,0.5517241
58,BS_01DQH017,7316-3299,chr14:24409238,NYNRIN,57523,Mutect2,GRCh38,chr14,24409238,24409238,⋯,NA,NA,,NA,,,NA,,,0.2470588
59,BS_01DQH017,7316-3299,chr4:56338715,AASDH,132949,Mutect2,GRCh38,chr4,56338715,56338715,⋯,NA,NA,,NA,,,NA,,,0.2575758


In [23]:
cbtn_h3f3a <- CBTN_data_maf_primary[which(CBTN_data_maf_primary$Hugo_Symbol == "H3F3A"),]
cbtn_idh1 <- CBTN_data_maf_primary[which(CBTN_data_maf_primary$Hugo_Symbol == "IDH1"),]

In [24]:
cbtn_h3f3a_k27 <- cbtn_h3f3a[which(grepl("K28M", cbtn_h3f3a$HGVSp_Short)),]
cbtn_h3f3a_g34 <- cbtn_h3f3a[which(grepl("G35", cbtn_h3f3a$HGVSp_Short)),]

In [25]:
k27_samples <- cbtn_h3f3a_k27$sample_id
g34_samples <- cbtn_h3f3a_g34$sample_id
idh1_samples <- cbtn_idh1$sample_id

In [26]:
metadata$tumour_subtype[metadata$sample_id %in% idh1_samples &
                        metadata$cohort == "pedHGG" & metadata$group == "CBTN"] <- "HGG_IDH1"
metadata$tumour_subtype[metadata$sample_id %in% g34_samples &
                        metadata$cohort == "pedHGG"& metadata$group == "CBTN"] <- "HGG_G34"
metadata$tumour_subtype[metadata$sample_id %in% k27_samples &
                        metadata$cohort == "pedHGG"& metadata$group == "CBTN"] <- "HGG_K27M"
metadata$tumour_subtype[is.na(metadata$tumour_subtype) &
                        metadata$cohort == "pedHGG"& metadata$group == "CBTN"] <- "HGG_WT"

### ICGC

Get DKFZ tumour subtypes from clinical metadata

In [27]:
metadata$tumour_subtype[metadata$cohort == "pedHGG" & metadata$group == "ICGC"] <- 
metadata$disease_type[metadata$cohort == "pedHGG" & metadata$group == "ICGC"]

In [28]:
metadata$tumour_subtype[grepl("HGGother_IDH", metadata$tumour_subtype)] <- "HGG_IDH1"
metadata$tumour_subtype[grepl("HGG_K27M", metadata$tumour_subtype)] <- "HGG_K27M"
metadata$tumour_subtype[grepl("HGGother_G34R", metadata$tumour_subtype)] <- "HGG_G34"
metadata$tumour_subtype[grepl("HGGother_WT", metadata$tumour_subtype)] <- "HGG_WT"

##  EPN subgroups

In [29]:
EPNs <- metadata[metadata$cohort == "EPN",]

In [30]:
table(EPNs$primary_site)


   Brain Stem- Midbrain/Tectum     Cerebellum/Posterior Fossa 
                             2                             20 
                  Frontal Lobe                 Occipital Lobe 
                             9                              4 
                 Parietal Lobe          Spinal Cord- Cervical 
                             5                              3 
Spinal Cord- Lumbar/Thecal Sac                      Spine NOS 
                             5                              1 
                 Temporal Lobe                     Ventricles 
                             2                             14 

In [31]:
epn_fossa <- EPNs$sample_id[grepl("Posterior Fossa", EPNs$primary_site)]
epn_spine <- EPNs$sample_id[grepl("Spin", EPNs$primary_site)]
epn_supratent <- EPNs$sample_id[grepl("Lobe", EPNs$primary_site) | grepl("Ventricle", EPNs$primary_site)]

In [32]:
metadata$tumour_subtype[metadata$sample_id %in% epn_fossa & metadata$cohort == "EPN"] <- "EPN_Fossa"
metadata$tumour_subtype[metadata$sample_id %in% epn_spine & metadata$cohort == "EPN"] <- "EPN_Spine"
metadata$tumour_subtype[metadata$sample_id %in% epn_supratent & metadata$cohort == "EPN"] <- "EPN_Supra"
metadata$tumour_subtype[is.na(metadata$tumour_subtype) & metadata$cohort == "EPN"] <- "EPN_ND"

# NBL subgroups

From TARGET portal

In [33]:
NBLs <- metadata[metadata$cohort == "NBL",]

In [34]:
NBL_manifest <- read.table(paste0(manifestpath, "TARGET_NBL_Discovery_ClinicalData_20170525.txt"),
                          sep = "\t", na.strings = "", header = TRUE)

In [35]:
colnames(NBL_manifest)[colnames(NBL_manifest) == "TARGET.USI"] <- "case_id"

In [36]:
NBL_manifest_subset <- NBL_manifest[NBL_manifest$case_id %in% NBLs$case_id,]

In [37]:
dim(NBL_manifest_subset)

[1] 149  32

In [38]:
NBLs_MYCstatus <- merge(NBLs, NBL_manifest_subset[, c("case_id", "MYCN.status")], by = "case_id")

In [39]:
dim(NBLs_MYCstatus)

[1] 149  24

In [40]:
table(NBLs_MYCstatus$MYCN.status)


    Amplified Not Amplified       Unknown 
           31           117             1 

In [41]:
head(NBLs_MYCstatus)

,case_id,aliquot_id,StromalScore,ImmuneScore,ESTIMATEScore,name,sample_id,gender,ethnicity,race,⋯,recurrence,disease_type,primary_site,group,cohort,sample_type,age_class,tumour_subtype,CRI_cluster,MYCN.status
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,TARGET-30-PAIFXV,TARGET-30-PAIFXV-01A-01R,-537.97880,483.8556,-54.1232,af82838c-9071-4dd7-8766-b818cf61cf82.bam,TARGET-30-PAIFXV-01A,Female,Unknown,White,⋯,1,MB_GR34,Abdomen,TARGET,NBL,Primary Tumor,Children,NA,NA,Not Amplified
2,TARGET-30-PAIPGU,TARGET-30-PAIPGU-01A-01R,-319.22630,1240.6492,921.4229,b1d954e7-21a0-441e-a9c7-47e6a6454625.bam,TARGET-30-PAIPGU-01A,Female,Unknown,White,⋯,0,MB_GR34,Adrenal gland,TARGET,NBL,Primary Tumor,NA,NA,1,Not Amplified
3,TARGET-30-PAISNS,TARGET-30-PAISNS-01A-01R,-899.30090,-128.2337,-1027.5347,7e145f1d-1807-4615-95b5-2684ccbcb348.bam,TARGET-30-PAISNS-01A,Male,Unknown,White,⋯,0,MB_GR34,Abdomen,TARGET,NBL,Primary Tumor,NA,NA,1,Not Amplified
4,TARGET-30-PAITCI,TARGET-30-PAITCI-01A-01R,-327.26820,200.6735,-126.5947,9704403e-aaf9-4a97-99bc-b2a9a9861767.bam,TARGET-30-PAITCI-01A,Male,Unknown,White,⋯,1,MB_GR34,Adrenal gland,TARGET,NBL,Primary Tumor,Infants,NA,1,Not Amplified
5,TARGET-30-PAIVHE,TARGET-30-PAIVHE-01A-01R,-15.87499,1033.6469,1017.7719,d8d30bd7-608d-4cbd-ba31-3f1fa6fefacf.bam,TARGET-30-PAIVHE-01A,Male,not hispanic or latino,White,⋯,1,MB_GR34,Adrenal gland,TARGET,NBL,Primary Tumor,Children,NA,2,Not Amplified
6,TARGET-30-PAIXIF,TARGET-30-PAIXIF-01A-01R,460.15050,978.1672,1438.3177,7c2b0108-1231-4543-a8bf-eb6644e76f41.bam,TARGET-30-PAIXIF-01A,Male,Unknown,Unknown,⋯,1,MB_GR34,Retroperitoneum,TARGET,NBL,Primary Tumor,Children,NA,NA,Not Amplified


In [42]:
nbl_myca <- NBLs_MYCstatus$case_id[NBLs_MYCstatus$MYCN.status == "Amplified"]
nbl_mycna <- NBLs_MYCstatus$case_id[NBLs_MYCstatus$MYCN.status == "Not Amplified"]

In [43]:
metadata$tumour_subtype[metadata$case_id %in% nbl_myca & metadata$cohort == "NBL"] <- "NBL_MYCA"
metadata$tumour_subtype[metadata$case_id %in% nbl_mycna & metadata$cohort == "NBL"] <- "NBL_MYCNA"
metadata$tumour_subtype[is.na(metadata$tumour_subtype)& metadata$cohort == "NBL"] <- "NBL_ND"

In [44]:
table(metadata$tumour_subtype[which(metadata$cohort == "NBL")])


 NBL_MYCA NBL_MYCNA    NBL_ND 
       31       117         3 

# LGG subgroups

Add BRAF status

In [45]:
load(file = paste0(datapath, "fusion/arriba_df.RData"))

remove low confidence

In [46]:
arriba_df_conf <- arriba_df[arriba_df$confidence != "low",]

In [47]:
braf_fusion_samples <- arriba_df_conf$sample_id[grepl("BRAF", arriba_df_conf$gene2) & grepl("KIAA1549", arriba_df_conf$gene1)]

In [48]:
cbtn_braf <- CBTN_data_maf_primary[which(CBTN_data_maf_primary$Hugo_Symbol == "BRAF"),]
cbtn_braf_v600e_samples <- cbtn_braf$sample_id[which(grepl("V600E", cbtn_braf$HGVSp_Short))]

In [49]:
ICGC_maf <- read.table(file = paste0(datapath, "snv/dkfz_mafKB.maf"),
                        sep = "\t", check.names = F, header = TRUE, comment.char = "",stringsAsFactors = FALSE)

In [50]:
ICGC_maf_nsyn <- ICGC_maf[!ICGC_maf$variant_classification %in% c("splicing", "unknown", "synonymous SNV"),]

In [51]:
colnames(ICGC_maf_nsyn)[colnames(ICGC_maf_nsyn) == "Tumor_Sample_Barcode"] <- "sample_id"

In [52]:
head(ICGC_maf_nsyn)

,sample_id,Hugo_Symbol,HGVSp_Short,variant_classification,mutation_effect,oncogenic,LEVEL_1,LEVEL_2A,LEVEL_2B,LEVEL_3A,LEVEL_3B,LEVEL_4,LEVEL_R1,LEVEL_R2,LEVEL_R3,Highest_level,citations
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<chr>,<chr>
1,H049-UV6K,PABPC1,p.E300X,stopgain SNV,,,NA,NA,,NA,,,NA,NA,NA,,
2,H049-UV6K,PABPC1,p.E345X,stopgain SNV,,,NA,NA,,NA,,,NA,NA,NA,,
3,H049-UV6K,PABPC1,p.E313X,stopgain SNV,,,NA,NA,,NA,,,NA,NA,NA,,
4,H049-UV6K,SPTBN2,p.G862E,nonsynonymous SNV,,,NA,NA,,NA,,,NA,NA,NA,,
5,H049-UV6K,SPTBN2,p.G862E,nonsynonymous SNV,,,NA,NA,,NA,,,NA,NA,NA,,
6,H049-UV6K,SPTBN2,p.G862E,nonsynonymous SNV,,,NA,NA,,NA,,,NA,NA,NA,,


In [53]:
dkfz_braf <- ICGC_maf_nsyn[which(ICGC_maf_nsyn$Hugo_Symbol == "BRAF"),]
dkfz_braf_v600e_samples <- dkfz_braf$sample_id[which(grepl("V600E", dkfz_braf$HGVSp_Short))]

In [54]:
braf_v600e_samples <- c(dkfz_braf_v600e_samples, cbtn_braf_v600e_samples)

In [55]:
braf_fusion_samples

[1] "7316-1081"  "7316-1090"  "7316-1134"  "7316-117"   "7316-119"  
  [6] "7316-120"   "7316-121"   "7316-1226"  "7316-127"   "7316-134"  
 [11] "7316-134"   "7316-146"   "7316-162"   "7316-1637"  "7316-1643" 
 [16] "7316-1646"  "7316-1648"  "7316-1664"  "7316-1668"  "7316-168"  
 [21] "7316-1683"  "7316-1710"  "7316-1710"  "7316-1711"  "7316-1779" 
 [26] "7316-1794"  "7316-1803"  "7316-1805"  "7316-1851"  "7316-1926" 
 [31] "7316-1928"  "7316-1942"  "7316-1952"  "7316-1953"  "7316-1958" 
 [36] "7316-1963"  "7316-1975"  "7316-1978"  "7316-20"    "7316-206"  
 [41] "7316-2099"  "7316-2167"  "7316-2172"  "7316-2184"  "7316-2197" 
 [46] "7316-2204"  "7316-222"   "7316-2291"  "7316-2305"  "7316-247"  
 [51] "7316-2572"  "7316-262"   "7316-2663"  "7316-2689"  "7316-2729" 
 [56] "7316-2755"  "7316-297"   "7316-3054"  "7316-3055"  "7316-3062" 
 [61] "7316-3068"  "7316-3140"  "7316-3154"  "7316-3169"  "7316-3203" 
 [66] "7316-3308"  "7316-332"   "7316-345"   "7316-346"   "7316-3491" 
 [71] "7316-3504"  "7316-3515"  "7316-3555"  "7316-3566"  "7316-3622" 
 [76] "7316-368"   "7316-370"   "7316-390"   "7316-391"   "7316-3919" 
 [81] "7316-393"   "7316-400"   "7316-436"   "7316-441"   "7316-459"  
 [86] "7316-460"   "7316-472"   "7316-475"   "7316-476"   "7316-487"  
 [91] "7316-495"   "7316-516"   "7316-562"   "7316-643"   "7316-666"  
 [96] "7316-733"   "7316-874"   "7316-891"   "7316-905"   "7316-924"  
[101] "7316-925"   "7316-938"   "7316-954"   "ICGC_PA100" "ICGC_PA103"
[106] "ICGC_PA107" "ICGC_PA108" "ICGC_PA109" "ICGC_PA11"  "ICGC_PA110"
[111] "ICGC_PA136" "ICGC_PA138" "ICGC_PA14"  "ICGC_PA147" "ICGC_PA148"
[116] "ICGC_PA150" "ICGC_PA157" "ICGC_PA157" "ICGC_PA162" "ICGC_PA165"
[121] "ICGC_PA4"   "ICGC_PA8"   "ICGC_PA87"  "ICGC_PA88"  "ICGC_PA9"  
[126] "ICGC_PA91"  "ICGC_PA93"  "ICGC_PA94"  "ICGC_PA95"  "ICGC_PA97"

In [56]:
metadata$tumour_subtype[metadata$sample_id %in% braf_fusion_samples & metadata$cohort == "pedLGG"] <- "BRAF_KIAA1549"
metadata$tumour_subtype[metadata$sample_id %in% braf_v600e_samples & metadata$cohort == "pedLGG"] <- "BRAF_p.V600E"
metadata$tumour_subtype[is.na(metadata$tumour_subtype)& metadata$cohort == "pedLGG"] <- "BRAF_WT"

In [57]:
table(metadata$tumour_subtype)


     ATRT_MYC      ATRT_SHH      ATRT_TYR BRAF_KIAA1549  BRAF_p.V600E 
            9             7             5           127            32 
      BRAF_WT     EPN_Fossa        EPN_ND     EPN_Spine     EPN_Supra 
          139            20             2             9            34 
      HGG_G34      HGG_K27M        HGG_WT       MB_GR34        MB_SHH 
            1            18            42           212            17 
       MB_WNT      NBL_MYCA     NBL_MYCNA        NBL_ND 
           11            31           117             3 

In [58]:
estimate_manifest_primary_clean <- metadata

In [59]:
save(estimate_manifest_primary_clean,
         file = paste0(datapath, "ESTIMATE/estimate_manifest_primary_clean_final.RData"))